## Importing Libraries

In [13]:
import json, random
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import faiss
import requests
import json
import time
import re

## Load Data

In [2]:
jsonl_path = "cars_RAG.jsonl"

cars = []
with open(jsonl_path, "r", encoding="utf-8") as f:
    for line in f:
        cars.append(json.loads(line))

len(cars), cars[0]


(985,
 {'id': 'car_0',
  'text': 'Mercedes-Benz | Mercedes-Benz CLA 200 Shooting Brake | First registration: 2017-09 | Accident-free: No | Mileage: 92,215 km | Fuel: Petrol | Power: 115.0 kW | Price: €18,999 | Seller: Kamux Auto GmbH (4.3⭐) | Description: 7G-DCT AMG Line+LED+NAVI',
  'meta': {'brand': 'Mercedes-Benz',
   'model': 'Mercedes-Benz CLA 200 Shooting Brake',
   'price_eur': 18999,
   'first_registration': '2017-09',
   'mileage_km': 92215,
   'car_link': 'https://suchen.mobile.de/fahrzeuge/details.html?id=442028837&cn=DE&dam=false&gn=Kiel%2C+Schleswig-Holstein&isSearchRequest=true&ll=54.322684%2C10.13586&od=up&rd=100&ref=srp&refId=692b2536-d65d-caf0-12e7-d05cb10a818e&s=Car&sb=rel&searchId=692b2536-d65d-caf0-12e7-d05cb10a818e&vc=Car',
   'accident_free': False,
   'car_age_years': 8,
   'mileage_category': '50k-100k',
   'power_kw': 115.0,
   'fuel_type': 'Petrol',
   'seller_name': 'Kamux Auto GmbH',
   'seller_rating': 4.3,
   'seller_reviews_count': 161}})

In [3]:
texts = [c["text"] for c in cars]
ids = [c["id"] for c in cars]

print("Sample text:", texts[0][:200])
print("Total cars:", len(texts))


Sample text: Mercedes-Benz | Mercedes-Benz CLA 200 Shooting Brake | First registration: 2017-09 | Accident-free: No | Mileage: 92,215 km | Fuel: Petrol | Power: 115.0 kW | Price: €18,999 | Seller: Kamux Auto GmbH 
Total cars: 985


## Embedding Model

In [4]:
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

embs = embed_model.encode(
    texts,
    normalize_embeddings=True
).astype("float32")

embs.shape


(985, 384)

## FAISS

In [5]:
dim = embs.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embs)

print("FAISS index built:", index.ntotal, "vectors")


FAISS index built: 985 vectors


## Cross Encoder

In [7]:
from sentence_transformers import CrossEncoder

cross_model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")


## Retrieve Function

In [6]:
def retrieve(query, top_k=100):
    q_emb = embed_model.encode([query], normalize_embeddings=True).astype("float32")
    D, I = index.search(q_emb, top_k)

    results = []
    for idx, score in zip(I[0], D[0]):
        item = cars[idx].copy()
        item["score"] = float(score)
        results.append(item)

    return results


## Rerank Function

In [8]:
def rerank(query, candidates, final_k=5):
    pairs = [(query, c["text"]) for c in candidates]   # query, candidate_text
    scores = cross_model.predict(pairs)

    for item, score in zip(candidates, scores):
        item["_cross_score"] = float(score)

    # sort by cross-encoder score
    ranked = sorted(candidates, key=lambda x: x["_cross_score"], reverse=True)
    return ranked[:final_k]


## Price Filter Extraction 

In [9]:
def extract_price_constraints(query):
    q = query.lower().replace(",", "")

    # BETWEEN range (15k to 25k, 15-25k)
    match = re.search(r'(\d+)\s*(k|000)?\s*[-toand]+\s*(\d+)\s*(k|000)?', q)
    if match:
        low = int(match.group(1)) * (1000 if match.group(2) else 1)
        high = int(match.group(3)) * (1000 if match.group(4) else 1)
        return {"type": "between", "min": low, "max": high}

    # LESS THAN / UNDER
    match = re.search(r'(less than|under|below|<)\s*(\d+)\s*(k|000)?', q)
    if match:
        val = int(match.group(2)) * (1000 if match.group(3) else 1)
        return {"type": "max", "value": val}

    # MORE THAN / ABOVE
    match = re.search(r'(more than|over|above|greater than|>|plus)\s*(\d+)\s*(k|000)?', q)
    if match:
        val = int(match.group(2)) * (1000 if match.group(3) else 1)
        return {"type": "min", "value": val}

    # AROUND / NEAR
    match = re.search(r'(around|near|approx)\s*(\d+)\s*(k|000)?', q)
    if match:
        center = int(match.group(2)) * (1000 if match.group(3) else 1)
        return {"type": "around", "center": center, "range": 3000}

    # Brief formats like 40k, <20k, >30k
    match = re.search(r'([<>])\s*(\d+)\s*(k|000)?', q)
    if match:
        val = int(match.group(2)) * (1000 if match.group(3) else 1)
        if match.group(1) == ">":
            return {"type": "min", "value": val}
        else:
            return {"type": "max", "value": val}

    # Single number like "20k budget"
    match = re.search(r'(\d+)\s*(k|000)?', q)
    if match:
        center = int(match.group(1)) * (1000 if match.group(2) else 1)
        return {"type": "around", "center": center, "range": 3000}

    return None


def apply_price_filter(candidates, rule):
    if not rule:
        return candidates

    t = rule["type"]

    if t == "min":  
        return [c for c in candidates if c["meta"]["price_eur"] >= rule["value"]]

    if t == "max":  
        return [c for c in candidates if c["meta"]["price_eur"] <= rule["value"]]

    if t == "between":
        return [
            c for c in candidates
            if rule["min"] <= c["meta"]["price_eur"] <= rule["max"]
        ]

    if t == "around":
        low = rule["center"] - rule["range"]
        high = rule["center"] + rule["range"]
        return [
            c for c in candidates
            if low <= c["meta"]["price_eur"] <= high
        ]

    return candidates

## OLLAMA Chat

In [10]:

OLLAMA_URL = "http://localhost:11434"
OLLAMA_MODEL = "qwen2.5:7b"

def ollama_chat(prompt, max_tokens=200, temperature=0.1):
    payload = {
        "model": OLLAMA_MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "stream": True,
        "max_tokens": max_tokens,
        "temperature": temperature
    }

    resp = requests.post(f"{OLLAMA_URL}/api/chat", json=payload, stream=True)
    resp.raise_for_status()

    full_text = ""
    for line in resp.iter_lines():
        if not line:
            continue
        try:
            data = json.loads(line.decode("utf-8"))
        except json.JSONDecodeError:
            continue
        if "message" in data:
            full_text += data["message"]["content"]

    return full_text


## Main ask Function

In [11]:
def generate_answer(query, reranked, top_k_for_llm=3):
    items = reranked[:top_k_for_llm]

    # compact context
    ctx_lines = []
    for i, r in enumerate(items, start=1):
        m = r["meta"]
        ctx_lines.append(
            f"{i}. {m['model']} | {m['price_eur']} EUR | "
            f"{m['mileage_km']} km | {m['fuel_type']} | {m['car_link']}"
        )

    context = "\n".join(ctx_lines)

    prompt = f"""
You are a car recommendation assistant.

User query:
{query}

Top candidate cars:
{context}

Give a highly accurate recommendation of 3 cars based on the user's query.
Format:
- 2–3 sentences of reasoning
- then list each recommended car
- show model, price, mileage, and LINK
- keep it concise and precise
"""
    return ollama_chat(prompt, max_tokens=400)


In [ ]:
def ask(query):
    t0 = time.time()
    retrieved = retrieve(query, top_k=100)
    price_rule = extract_price_constraints(query)
    if price_rule:
        retrieved = apply_price_filter(retrieved, price_rule)

    if not retrieved:
        return {
            "reasoning": f"No cars match your filters: {query}",
            "cars": []
        }
    t1 = time.time()
    reranked = rerank(query, retrieved, final_k=5)
    t2 = time.time()
    answer = generate_answer(query, reranked, top_k_for_llm=3)
    t3 = time.time()

    print(f"Retrieve: {t1-t0:.3f}s | Rerank: {t2-t1:.3f}s | LLM: {t3-t2:.3f}s | Total: {t3-t0:.3f}s\n")
    return answer

query = "looking for a accident free petrol car around 30000"
print(ask(query))


Retrieve: 0.020s | Rerank: 0.511s | LLM: 188.488s | Total: 189.020s

Based on your criteria of an accident-free petrol car within 30,000 EUR, here are three highly recommended options:

1. Mercedes-Benz E 200 | 28,990 EUR | 87,000 km | [Link](https://suchen.mobile.de/fahrzeuge/details.html?id=441413208&cn=DE&dam=false&gn=Kiel%2C+Schleswig-Holstein&isSearchRequest=true&ll=54.322684%2C10.13586&od=up&pageNumber=11&rd=100&ref=srp&refId=220fc038-2d42-22b2-dbe7-552aeafdbe22&s=Car&sb=rel&searchId=220fc038-2d42-22b2-dbe7-552aeafdbe22&vc=Car)
2. Mercedes-Benz A 250 | 28,990 EUR | 64,608 km | [Link](https://suchen.mobile.de/fahrzeuge/details.html?id=430648010&cn=DE&dam=false&gn=Kiel%2C+Schleswig-Holstein&isSearchRequest=true&ll=54.322684%2C10.13586&od=up&pageNumber=46&rd=100&ref=srp&refId=3be2404f-138f-1e4a-81e4-f018cc59b040&s=Car&sb=rel&searchId=3be2404f-138f-1e4a-81e4-f018cc59b040&vc=Car)
3. Mercedes-Benz E 200 | 32,990 EUR | 82,000 km | [Link](https://suchen.mobile.de/fahrzeuge/details.html?i


# ----Sample Queries-------
1. Recommend a low-mileage VW Golf model.
2. Find an accident-free petrol car around 20k euros.
3. Show me the cheapest cars that have a power output greater than 150 kW.
4. List the details for petrol car that is priced around €40,000.
5. Find the best-rated, accident-free BMW car listed under €30,000 that has low mileage
